In [ ]:
import os
import sys
my_path = "/kaggle/input/mlsecu" # path to utils folder
sys.path.append(my_path)
from mlsecu.anomaly_detection_use_case import *
from mlsecu.data_exploration_utils import *
from mlsecu.data_preparation_utils import *
import pandas as pd

### Loading Dataset

In [ ]:
TRAIN_TRANSACTION_FILE_PATH = os.path.join('/kaggle/input/ieee-fraud-detection','train_transaction.csv')

In [ ]:
transaction_df = pd.read_csv(TRAIN_TRANSACTION_FILE_PATH)

In [ ]:
transaction_df.head()

In [ ]:
non_fraud_df = transaction_df[transaction_df['isFraud'] == 0][:9700]
reduced_transaction_df = transaction_df[transaction_df['isFraud'] == 1][:300]

In [ ]:
non_fraud_df.head()

In [ ]:
reduced_transaction_df.head()

In [ ]:
os.path.exists(path)

In [ ]:
! [ -e /kaggle/input/mlsecu/mlsecu ] 